<a href="https://colab.research.google.com/github/Saharg8/BI-Project-ChicagoCrimes/blob/main/BI_STTM_D_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install jsonpath_ng

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.6 MB/s eta 0:00:00


In [22]:
import os
import csv
import json
import numpy as np
import pandas as pd
import datetime
import sqlite3
import collections
import functools as ft
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from jsonpath_ng import parse
from enum import Enum
%matplotlib inline

# Source to Target Mapping - OOP Approach for Single Table

### Data

In [23]:
data = pd.read_csv('Areas Codes and Names.csv', encoding='latin1')
data.to_json('Areas Codes and Names.json')
json_data = data.astype(str).to_dict(orient='records')

# json_data
# json_data[:3]

###  Abstract Base Calss (ABC) for mutual methods

In [24]:
class Interface(ABC):

    @abstractmethod
    def get_data_by_field(self, field_name):
        """Fetch the data by given feild name """

    @abstractmethod
    def get_data_by_id(self, id):
        """Fetch the data by given ID  """

    @abstractmethod
    def get(self):
        """Fetch all data """

### Transform Operations
inherithed from Enum - class that automatic enumrate the variables

In [25]:
class TransformMask(Enum):
    # add here any masks you want
    CLEAN_STRING = ".strip().lower()"
    CAPITAL_LETTER = ".strip().lower().title()"


### Database Class - Define Common Properties for Source, Target, Mapping

In [26]:
class Database:
    def __init__(self):
        pass

    # built-in function that creates and returns a property object
    # get data by: get_data_source_target_mapping.get(dict_key)
    @property
    def get_data_source_target_mapping(self):
        return {
            "source": [
                {
                    "id": 1,
                    "source_field_name": "number",
                    "source_field_mapping": "$.number",
                    "source_field_type": "str",
                    "is_required": True,

                },
                {
                    "id": 2,
                    "source_field_name": "Name",
                    "source_field_mapping": "$.Name",
                    "source_field_type": "str",
                    "is_required": True,

                },
                {
                    "id": 3,
                    "source_field_name": "Population2020",
                    "source_field_mapping": "$.Population2020",
                    "source_field_type": "str",
                    "is_required": True,

                },

            ],
            "destination": [
                {
                    "id": 1,
                    "destination_field_name": "AreaNumber",
                    "destination_field_mapping": "AreaNumber",
                    "destination_field_type": "str",
                    "default_value": "n/a",
                },
                {
                    "id": 2,
                    "destination_field_name": "AreaName",
                    "destination_field_mapping": "AreaName",
                    "destination_field_type": "str",
                    "default_value": "n/a"
                },
                {
                    "id": 3,
                    "destination_field_name": "Population2020",
                    "destination_field_mapping": "Population2020",
                    "destination_field_type": "int",
                    "default_value": "0"
                },
            ],
            "transform": [
                {
                    "id": 1,
                    "transform_mask": 'CAPITAL_LETTER'
                },
                {
                    "id": 2,
                    "transform_mask": 'CLEAN_STRING'
                }
            ],
            "mapping": [
                {
                    "id": 1,
                    "mapping_source": 1,
                    "mapping_destination": 1
                },
                {
                    "id": 2,
                    "mapping_source": 2,
                    "mapping_destination": 2,
                    "mapping_transform": 1,
                    "mapping_transform": 2
                },
                {
                    "id": 3,
                    "mapping_source": 3,
                    "mapping_destination": 3
                },
            ]
        }

### Source class

Inherited from Interface for the common methods and from Database for common variables

In [27]:
class Source(Interface, Database):
    def __init__(self):
        Database.__init__(self)

    # should be implemented - inherited from Interface
    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("source")

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id") == self.id:
                return x
        return None

### Target class

Inherited from Interface for the common methods and from Database for common variables

In [28]:
class Target(Interface, Database):

    def __init__(self):
        Database.__init__(self)

    # should be implemented - inherited from Interface
    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("destination")

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

### Transform Class

Inherited from Interface for the common methods and from Database for common variables

In [29]:
class Transform(Interface, Database):

    def __init__(self):
        Database.__init__(self)

    # should be implemented - inherited from Interface
    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("transform", [])

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

### Mapping class

Inherited from Interface for the common methods and from Database for common variables

In [30]:
class Mappings(Interface, Database):

    def __init__(self):
        Database.__init__(self)

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("mapping")

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

    def get_data_by_field(self, field_name):
        return None

### Format Class - JSON

Search the source data value inside a JSON file

In [31]:
class JsonQuery:
    def __init__(self, json_path, json_data):
        self.json_path = json_path
        self.json_data = json_data

    def get(self):
        jsonpath_expression = parse(self.json_path)
        match = jsonpath_expression.find(self.json_data)
        source_data_value = match[0].value
        return source_data_value

### Combine it All - STTM

In [32]:
class STTM:
    def __init__(self, input_json):
        self.input_json = input_json
        self.mapping_instance = Mappings()
        self.source_instance = Source()
        self.destination_instance = Target()
        self.transform_instance = Transform()
        self.look_up_mask = {i.name: i.value for i in TransformMask}
        self.json_data_transformed = {}

    def _get_mapping_data(self):
        return self.mapping_instance.get

    def _get_mapping_source_data(self):
        return self.source_instance.get

    def get_transformed_data(self):

        for mappings in self._get_mapping_data():

            """fetch the source mapping """
            mapping_source_id = mappings.get("mapping_source")
            mapping_destination_id = mappings.get("mapping_destination")
            mapping_transform_id = mappings.get("mapping_transform")

            mapping_source_data = self.source_instance.get_data_by_id(id=mapping_source_id)
            transform_data = self.transform_instance.get_data_by_id(id=mapping_transform_id)

            """Fetch Source  field Name"""
            source_field_name = mapping_source_data.get("source_field_name")

            """if field given is not present incoming json """
            if source_field_name not in self.input_json.keys():
                if mapping_source_data.get("is_required"):
                    raise Exception(
                        "Alert ! Field {} is not present in JSON please FIX mappings ".format(source_field_name))
                else:
                    pass

            else:
                source_data_value = JsonQuery(
                    json_path=mapping_source_data.get("source_field_mapping"),
                    json_data=self.input_json
                ).get()

                """check the data type for source if matches with what we have """
                if mapping_source_data.get("source_field_type") != type(source_data_value).__name__:
                    if source_data_value is not None:
                        _message = (
                            "Alert ! Source Field :{} Datatype has changed from {} to {} ".format(source_field_name,
                                                                                                  mapping_source_data.get(
                                                                                                      "source_field_type"),
                                                                                                  type(
                                                                                                      source_data_value).__name__))
                        print(_message)
                        raise Exception(_message)

                """Query and fetch the Destination | target """
                destination_mappings_json_object = self.destination_instance.get_data_by_id(
                    id=mappings.get("mapping_destination"))

                destination_field_name = destination_mappings_json_object.get("destination_field_name")
                destination_field_type = destination_mappings_json_object.get("destination_field_type")

                dtypes = [str, float, list, int, set, dict]

                for dtype in dtypes:

                    """Datatype Conversion """
                    if destination_field_type == str(dtype.__name__):

                        """is source is none insert default value"""
                        if source_data_value is None:
                            self.json_data_transformed[destination_field_name] = dtype.__call__(
                                destination_mappings_json_object.get("default_value")
                            )

                        else:
                            """check if you have items to transform"""
                            if transform_data is not None:
                                """ check for invalid mask name """
                                if transform_data.get("transform_mask") not in list(self.look_up_mask.keys()):
                                    raise Exception(
                                        f"Specified Transform {transform_data.get('transform_mask')} is not available please select from following Options :{list(self.look_up_mask.keys())}")
                                else:
                                    mask_apply = self.look_up_mask.get(transform_data.get("transform_mask"))
                                    converted_dtype = dtype.__call__(source_data_value)
                                    mask = f'converted_dtype{mask_apply}'
                                    curated_value = eval(mask)
                                    self.json_data_transformed[destination_field_name] = curated_value

                            else:
                                self.json_data_transformed[destination_field_name] = dtype.__call__(source_data_value)

        return self.json_data_transformed

In [33]:
transformed_data = []

for item in json_data:
    helper = STTM(input_json=item)
    response = helper.get_transformed_data()
    transformed_data.append(response)
    print(response)



{'AreaNumber': '1', 'AreaName': 'rogers park', 'Population2020': 55628}
{'AreaNumber': '2', 'AreaName': 'west ridge', 'Population2020': 77122}
{'AreaNumber': '3', 'AreaName': 'uptown', 'Population2020': 57182}
{'AreaNumber': '4', 'AreaName': 'lincoln square', 'Population2020': 40494}
{'AreaNumber': '5', 'AreaName': 'north center', 'Population2020': 35114}
{'AreaNumber': '6', 'AreaName': 'lake view', 'Population2020': 103050}
{'AreaNumber': '7', 'AreaName': 'lincoln park', 'Population2020': 70492}
{'AreaNumber': '8', 'AreaName': 'near north side', 'Population2020': 105481}
{'AreaNumber': '9', 'AreaName': 'edison park', 'Population2020': 11525}
{'AreaNumber': '10', 'AreaName': 'norwood park', 'Population2020': 38303}
{'AreaNumber': '11', 'AreaName': 'jefferson park', 'Population2020': 26216}
{'AreaNumber': '12', 'AreaName': 'forest glen', 'Population2020': 19596}
{'AreaNumber': '13', 'AreaName': 'north park', 'Population2020': 17559}
{'AreaNumber': '14', 'AreaName': 'albany park', 'Popul

In [34]:
pd.DataFrame(transformed_data).to_csv('Areas Codes and Names - After STTM.csv', index=False)